In [4]:
import camelot
import pandas
import os
import copy
from datetime import datetime, timedelta
import feedparser
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from twitter_post import post
import re
import PyPDF2
from pathlib import Path

In [12]:
pdf_file_path = Path.cwd().parent.joinpath("data", "202105.pdf")

print(f"generating {pdf_file_path}")


pdf_read = PyPDF2.PdfFileReader(pdf_file_path.__str__())
num_page = pdf_read.numPages

if num_page > 10:
    page_list = list(range(10, num_page, 10))
else:
    page_list = [num_page]
init = 1
dfs_lap = []
for num in page_list:
    if num == page_list[-1]:
        num = "end"
    tbls = camelot.read_pdf(pdf_file_path.__str__(), pages=f'{init}-{num}')
    if num != "end":
        init = num + 1

    dfs = []
    for table in tbls:
        df = table.df
        dfs.append(df)
    df_all = pandas.concat(dfs)
    dfs_lap.append(df_all)
df_total = pandas.concat(dfs_lap)

generating C:\Users\shuny\Documents\twitter_aichi_covid19\data\202105.pdf


In [15]:
df_total.to_pickle("database.zip")

In [16]:
df_all.columns = df_all.iloc[0, :]


,0,1,2,3,4,5,6
0,No,発表日,年代・性別,国籍,住居地,接触状況,備考
1,1,1月26日,40代男性,中国,中国武漢市,中国,本県発表1
2,2,1月28日,40代男性,中国,中国武漢市,中国,本県発表2
3,3,2月14日,60代男性,日本,名古屋市,アメリカ,名古屋市発表1
4,4,2月15日,60代女性,日本,名古屋市,No.3と接触,名古屋市発表2
...,...,...,...,...,...,...,...
20,47344,5月31日,70代女性,,名古屋市,No.42817と接触,名古屋市発表20612
21,47345,5月31日,40代男性,,名古屋市,,名古屋市発表20613
22,47346,5月31日,30代女性,,名古屋市,神奈川県事例と接触,名古屋市発表20614
23,47347,5月31日,50代女性,,名古屋市,,名古屋市発表20615


In [17]:


df_all = copy.deepcopy(df_total)

df_all.columns = df_all.iloc[0, :]
df_all = df_all.sort_values("No")
df_all = df_all[df_all["No"] != "No"]
df_all = df_all.set_index("No")

In [78]:
# 発表日が欠番なものを削除
df_all = df_all[~df_all["発表日"].str.contains("欠番")]

# インデックスが修正なものを削除
df_all = df_all[~df_all.index.str.contains("修正")]

issued_date = [_.split("\n") for _ in df_all["発表日"].to_list()]

# num_issued_date = [len(_.split("\n")) for _ in df_all["発表日"].to_list()]
refined_list = []
for i, _ in enumerate(issued_date):
    if len(_) > 1:
        
        new_list = _+ [elem for elem in df_all.iloc[i, :].to_list() if (elem!="")&(elem!="\n".join(_))]
        new_list.insert(2, "")
        if len(new_list) == 5:
            new_list.insert(4, "")
        refined_list.append(new_list)
    else:
        refined_list.append(df_all.iloc[i, :].to_list())

for i in range(len(df_all.index)):
    df_all.iloc[i,:] = refined_list[i]



# num_issued_date = [len(_.split("\n")) for _ in df_all["発表日"].to_list()]
# for i, _ in enumerate(num_issued_date):
#     if _ >1:
#         print(i)

# num_issued_attrib = [len(_.split("\n")) for _ in df_all["年代・性別"].to_list()]
# for i, _ in enumerate(num_issued_attrib):
#     if _ >1:
#         print(i)

In [104]:
df_all.index[4]

'10000'

In [106]:
new_index = []
new_date = []
for i, _ in enumerate(df_all.index.to_list()):
    if not _.isdigit():
        index, remaining = _.split()
        remaining = remaining.replace("※", "")
        new_index.append(int(index))
        new_date.append(remaining)
    else:
        new_index.append(int(_))
        new_date.append(df_all.iloc[i, :]["発表日"])
df_all.index = new_index
df_all["発表日"] = new_date
#         df_all.index[i] = index
#         df_all.iloc[i, :] = [remaining] + df_all.iloc[i, :].to_list()[1:]
#         print(df_all.iloc[i, :])

In [108]:
df_all = df_all.sort_index()

In [115]:
is_2020 = True
months = [int(_.split("月")[0]) for _ in df_all["発表日"].to_list()]
for i, month in enumerate(months): 
    if month > months[i+1]+10:
        year_start = i
        break

df_all["発表日"] = pandas.to_datetime(
    ['2021年' + _ if no > year_start else '2020年' + _ for no, _ in enumerate(df_all['発表日'])], format='%Y年%m月%d日')

In [117]:
df_all.to_pickle("202105.zip")

In [72]:
"\n".join(['4月29日', '40代男性', '岡崎市', 'No.32610と接触', '岡崎市発表1204']) == '4月29日\n40代男性\n岡崎市\nNo.32610と接触\n岡崎市発表1204'

True

In [76]:
test = ['4月29日', '40代男性', '岡崎市', 'No.32610と接触', '岡崎市発表1204']
test.insert(2, "")
test

['4月29日', '40代男性', '', '岡崎市', 'No.32610と接触', '岡崎市発表1204']

In [41]:
exceptions2 = [index for index, _date in zip(df_all.index, df_all["発表日"]) if "欠番" in _date]
exceptions = [index for index in df_all.index if "陽性者公表数の修正" in index ]
exceptions += exceptions2
df_all2 = df_all.drop(exceptions, axis=0)

In [42]:
new_index = []
new_date = []
for no in df_all2.index:
    if df_all2.at[no, "発表日"] == "":
        _no, _date = no.split()
    else:
        _no, _date = no, df_all2.at[no, "発表日"]
    new_index.append(int(_no))
    new_date.append(_date.replace("※", ""))
df_all2.index = new_index
df_all2.loc[:, "発表日"] = new_date

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [indexing.py:1843]


In [43]:
df_all2[df_all2["発表日"] == ""]

,発表日,年代・性別,国籍,住居地,接触状況,備考


In [44]:
df_all2.index = [int(_) for _ in df_all2.index]

In [45]:
for ind in df_all2.index:
    try:
        int(ind)
    except:
        print(df_all2.loc[ind, :])

In [8]:
list0 = df_all2.loc[6951,"発表日"].split("\n")
list0.insert(2,"")
list0
df_all2.loc[6951,:] = list0

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [indexing.py:1637]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [indexing.py:692]


In [47]:
df_all2[df_all2["発表日"]==""]

,発表日,年代・性別,国籍,住居地,接触状況,備考


,発表日,年代・性別,国籍,住居地,接触状況,備考
17184,1月4日,30代女性,,,,岡崎市発表575
26157,3月6日,60代男性,,,沖縄県事例と接触,岡崎市発表961
33012,4月29日\n40代男性\n岡崎市\nNo.32610と接触\n岡崎市発表1204,,,,,
41824,5月19日\n30代女性\n大治町,,,,,本県発表17466
41825,5月19日\n40代男性\nあま市,,,,,本県発表17467
41826,5月19日\n40代男性\nあま市,,,,,本県発表17468
42153,5月19日\n30代男性\n名古屋市,,,,,名古屋市発表18299
42154,5月19日\n90代女性\n名古屋市\nNo.39616と接触,,,,,名古屋市発表18300
42187,5月19日\n20代女性\n名古屋市,,,,,名古屋市発表18333
6951,11月10日\n10代女性\n春日井市\nNo.6555と接触\n本県発表2532,,,,,


In [46]:
df_all2[df_all2["年代・性別"]==""]

,発表日,年代・性別,国籍,住居地,接触状況,備考
33012,4月29日\n40代男性\n岡崎市\nNo.32610と接触\n岡崎市発表1204,,,,,
41824,5月19日\n30代女性\n大治町,,,,,本県発表17466
41825,5月19日\n40代男性\nあま市,,,,,本県発表17467
41826,5月19日\n40代男性\nあま市,,,,,本県発表17468
42153,5月19日\n30代男性\n名古屋市,,,,,名古屋市発表18299
42154,5月19日\n90代女性\n名古屋市\nNo.39616と接触,,,,,名古屋市発表18300
42187,5月19日\n20代女性\n名古屋市,,,,,名古屋市発表18333
6951,11月10日\n10代女性\n春日井市\nNo.6555と接触\n本県発表2532,,,,,


In [14]:
df_all2 = df_all2.sort_index()

In [31]:
for n,_ in enumerate(df_all2["発表日"]):
    if len(_)>6:
        print(n,_,df_all2.index[n])

33000 4月29日
40代男性
岡崎市
No.32610と接触
岡崎市発表1204 33012


In [36]:
df_all2

,発表日,年代・性別,国籍,住居地,接触状況,備考
1,1月26日,40代男性,中国,中国武漢市,中国,本県発表1
2,1月28日,40代男性,中国,中国武漢市,中国,本県発表2
3,2月14日,60代男性,日本,名古屋市,アメリカ,名古屋市発表1
4,2月15日,60代女性,日本,名古屋市,No.3と接触,名古屋市発表2
5,2月16日,60代男性,日本,尾張地方,No.4と接触,本県発表3
...,...,...,...,...,...,...
33485,4月30日,10歳未満女性,,名古屋市,No.31959と接触,名古屋市発表15058
33486,4月30日,20代男性,,名古屋市,No.31357と接触,名古屋市発表15059
33487,4月30日,40代女性,,名古屋市,No.30333と接触,名古屋市発表15060
33488,4月30日,30代男性,,千葉市,,名古屋市発表15061


In [38]:
l33012 = df_all2.loc[33012,"発表日"].split("\n")
l33012.insert(2,"")
l33012
df_all2.loc[33012,:]=l33012

In [39]:
df_all2.loc[33010:33015, :]

,発表日,年代・性別,国籍,住居地,接触状況,備考
33010,4月29日,40代男性,,岡崎市,No.32610と接触,岡崎市発表1202
33012,4月29日,40代男性,,岡崎市,No.32610と接触,岡崎市発表1204
33014,4月29日,60代女性,,岡崎市,No.32610と接触,岡崎市発表1206
33015,4月29日,70代女性,,岡崎市,No.32610と接触,岡崎市発表1207


In [40]:

df_all2["発表日"] = pandas.to_datetime(
    ['2021年' + _ if no > 16577 else '2020年' + _ for no, _ in zip(df_all2.index, df_all2['発表日'])], format='%Y年%m月%d日')

In [41]:

df_all2.to_pickle(f"{os.path.splitext(pdf_file_path)[0]}.zip")

In [113]:
pandas.read_pickle("database.zip")[27300:]

,発表日,年代・性別,国籍,住居地,接触状況,備考
27308,2021-03-31,60代女性,,一宮市,,本県発表11298
27309,2021-03-31,70代女性,,瀬⼾市,,本県発表11299
27310,2021-03-31,40代女性,,瀬⼾市,No.27306と接触,本県発表11300
27311,2021-03-31,40代女性,,尾張旭市,,本県発表11301
27312,2021-03-31,30代女性,,日進市,,本県発表11302
...,...,...,...,...,...,...
27481,2021-04-01,50代女性,,名古屋市,,名古屋市発表12644
27482,2021-04-01,40代男性,,名古屋市,,名古屋市発表12645
27483,2021-04-01,30代男性,,名古屋市,No.26999と接触,名古屋市発表12646
27484,2021-04-01,80代男性,,名古屋市,No.26999と接触,名古屋市発表12647
